<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/Utility)250515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 0. 기본 설정 및 유틸리티 함수 ---
AIR_DENSITY_KG_M3 = 1.2
AIR_SPECIFIC_HEAT_J_KG_K = 1005
WATER_LATENT_HEAT_J_KG = 2260000

CHILLER_CAPACITY_KW = {'low_temp': 200, 'high_temp': 250}
COOLING_TOWER_CAPACITY_KW = 500

NUM_CHILLERS_TOTAL = {'low_temp': 8, 'high_temp': 8}
NUM_CTS_TOTAL = 9
NUM_OAC_UNITS = 14

def calculate_enthalpy(temperature_c, relative_humidity_percent, pressure_pa=101325):
    P_ws = 611.21 * np.exp((18.678 - temperature_c / 234.5) * (temperature_c / (257.14 + temperature_c)))
    P_w = (relative_humidity_percent / 100) * P_ws

    denominator_mixing_ratio = pressure_pa - P_w
    mixing_ratio = np.where(np.abs(denominator_mixing_ratio) < 1e-9, 0, 0.622 * P_w / denominator_mixing_ratio)

    enthalpy = (1.006 * temperature_c + mixing_ratio * (2501 + 1.86 * temperature_c)) * 1000
    return enthalpy

def calculate_absolute_humidity(temperature_c, relative_humidity_percent, pressure_pa=101325):
    P_ws = 611.21 * np.exp((18.678 - temperature_c / 234.5) * (temperature_c / (257.14 + temperature_c)))
    P_w = (relative_humidity_percent / 100) * P_ws

    denominator_mixing_ratio = pressure_pa - P_w
    mixing_ratio = np.where(np.abs(denominator_mixing_ratio) < 1e-9, 0, 0.622 * P_w / denominator_mixing_ratio)
    return mixing_ratio

def calculate_rh_from_abs_humidity(temperature_c, absolute_humidity_kg_kg, pressure_pa=101325):
    # absolute_humidity_kg_kg가 배열일 경우, 요소별로 0 미만인 값을 0으로 클리핑
    abs_hum_corrected = np.where(absolute_humidity_kg_kg < 0, 0, absolute_humidity_kg_kg)

    P_ws = 611.21 * np.exp((18.678 - temperature_c / 234.5) * (temperature_c / (257.14 + temperature_c)))

    denominator_pw = 0.622 + abs_hum_corrected
    P_w = np.where(np.abs(denominator_pw) < 1e-9, 0, abs_hum_corrected * pressure_pa / denominator_pw)

    relative_humidity_percent = np.where(np.abs(P_ws) < 1e-9, 0, (P_w / P_ws) * 100)

    return np.clip(relative_humidity_percent, 0, 100)

def calculate_dew_point(temperature_c, relative_humidity_percent):
    a = 17.625; b = 243.04
    rh_clipped = np.clip(relative_humidity_percent, 1e-5, 100) # log(0) 방지
    alpha = np.log(rh_clipped / 100) + (a * temperature_c) / (b + temperature_c)

    denominator_dew_point = a - alpha
    # 분모가 0에 가까운 경우 (포화 상태 근사), 이슬점은 현재 온도와 같다고 처리
    dew_point_c = np.where(np.abs(denominator_dew_point) < 1e-9, temperature_c, (b * alpha) / denominator_dew_point)
    return dew_point_c

# OAC 코일 통과 후 상태 계산 (반환값: 출구온도, 출구절대습도, 출구상대습도, 코일부하W)
def simulate_oac_coil_process(temp_in_c, abs_hum_in_kg_kg, sp_temp_c, opening_rate, coil_type, fan_cmh):
    effectiveness = 0.8
    temp_out_c = temp_in_c
    abs_hum_out_kg_kg = abs_hum_in_kg_kg
    load_watts = 0
    mass_flow_kg_s = (fan_cmh / 3600) * AIR_DENSITY_KG_M3

    if mass_flow_kg_s <= 0 or opening_rate <= 0:
        rh_out_percent = calculate_rh_from_abs_humidity(temp_out_c, abs_hum_out_kg_kg)
        return temp_out_c, abs_hum_out_kg_kg, rh_out_percent, load_watts

    if coil_type in ['예열', '승온']:
        if sp_temp_c > temp_in_c: # 이 조건은 스칼라 비교, vectorize 내부에서는 각 요소별로 처리됨
            delta_t_max = sp_temp_c - temp_in_c
            temp_out_c = temp_in_c + effectiveness * opening_rate * delta_t_max
            load_watts = mass_flow_kg_s * AIR_SPECIFIC_HEAT_J_KG_K * (temp_out_c - temp_in_c)
    elif coil_type in ['예냉', '냉각']:
        if sp_temp_c < temp_in_c:
            delta_t_max = temp_in_c - sp_temp_c
            temp_reduction = effectiveness * opening_rate * delta_t_max
            temp_out_c = temp_in_c - temp_reduction
            load_watts = mass_flow_kg_s * AIR_SPECIFIC_HEAT_J_KG_K * (temp_in_c - temp_out_c)

    load_watts = np.maximum(0, load_watts) # np.where 대신 np.maximum 사용 가능
    rh_out_percent = calculate_rh_from_abs_humidity(temp_out_c, abs_hum_out_kg_kg)
    return temp_out_c, abs_hum_out_kg_kg, rh_out_percent, load_watts

# OAC 가습 코일 통과 후 상태 계산 (반환값: 출구온도, 출구절대습도, 출구상대습도, 코일부하W)
def simulate_oac_humidifier_process(temp_in_c, abs_hum_in_kg_kg, sp_rh_percent, opening_rate, fan_cmh):
    temp_out_c = temp_in_c
    abs_hum_out_kg_kg = abs_hum_in_kg_kg
    load_watts = 0
    mass_flow_kg_s_air = (fan_cmh / 3600) * AIR_DENSITY_KG_M3

    if mass_flow_kg_s_air > 0 and opening_rate > 0:
        target_abs_hum_kg_kg = calculate_absolute_humidity(temp_in_c, sp_rh_percent)
        # np.where를 사용하여 조건부로 로직 실행 (vectorize 내부에서 요소별 처리)
        condition = target_abs_hum_kg_kg > abs_hum_in_kg_kg

        water_to_add_per_kg_air = np.where(condition, (target_abs_hum_kg_kg - abs_hum_in_kg_kg) * opening_rate, 0)
        water_added_kg_s = water_to_add_per_kg_air * mass_flow_kg_s_air

        abs_hum_out_kg_kg_if_humidified = abs_hum_in_kg_kg + water_added_kg_s / np.where(mass_flow_kg_s_air > 1e-9, mass_flow_kg_s_air, 1e-9)
        abs_hum_out_kg_kg = np.where(condition, abs_hum_out_kg_kg_if_humidified, abs_hum_in_kg_kg)

        load_watts = np.where(condition, water_added_kg_s * WATER_LATENT_HEAT_J_KG, 0)

    load_watts = np.maximum(0, load_watts)
    rh_out_percent = calculate_rh_from_abs_humidity(temp_out_c, abs_hum_out_kg_kg)
    return temp_out_c, abs_hum_out_kg_kg, rh_out_percent, load_watts

# --- 1. 더미 데이터 생성 ---
def generate_dummy_data(start_date="2024-01-01", periods=24*3, freq='60min', num_oac_total_units=NUM_OAC_UNITS):
    np.random.seed(42)
    datetime_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    num_samples = len(datetime_index)
    data = pd.DataFrame(index=datetime_index)

    day_of_year = data.index.dayofyear; hour_of_day = data.index.hour
    base_temp = 15; seasonal_amp = 15; daily_amp = 3
    data['외기_온도'] = base_temp + seasonal_amp*np.sin(2*np.pi*(day_of_year-90)/365) + daily_amp*np.sin(2*np.pi*hour_of_day/24) + np.random.normal(0,1.5,num_samples)
    data['외기_습도'] = np.clip(60-(data['외기_온도']-base_temp)*1.0 + 15*np.cos(2*np.pi*(day_of_year-180)/365) + np.random.normal(0,10,num_samples),20,95)

    # OAC Setpoints
    data['OAC_예열_set_point'] = 18.0 + np.random.uniform(-1,1,num_samples)
    data['OAC_예냉_set_point'] = 19.0 + np.random.uniform(-1,1,num_samples)
    data['OAC_냉각_set_point'] = 12.0 + np.random.uniform(-1,1,num_samples)
    data['OAC_승온_set_point'] = 22.0 + np.random.uniform(-1,1,num_samples)
    data['OAC_가습_set_point'] = 50.0 + np.random.uniform(-5,5,num_samples)
    data['OAC_FAN_Hz'] = 50 + np.random.uniform(-5,5,num_samples)
    oac_fan_cmh_values = data['OAC_FAN_Hz'] * 1000

    current_temp_c_arr = data['외기_온도'].values
    current_abs_hum_arr = np.vectorize(calculate_absolute_humidity)(data['외기_온도'].values, data['외기_습도'].values)

    oac_precool_load_kw_series = pd.Series(np.zeros(num_samples), index=data.index, dtype=float)
    oac_cooling_load_kw_series = pd.Series(np.zeros(num_samples), index=data.index, dtype=float)

    for coil_name_prefix, sp_col_name in [('예열', 'OAC_예열_set_point'), ('예냉', 'OAC_예냉_set_point'), ('냉각', 'OAC_냉각_set_point'), ('승온', 'OAC_승온_set_point')]:
        if coil_name_prefix in ['예열', '승온']:
            opening_rate_arr = np.clip((data[sp_col_name].values - current_temp_c_arr) / 10, 0, 1) * (current_temp_c_arr < data[sp_col_name].values)
        else:
            opening_rate_arr = np.clip((current_temp_c_arr - data[sp_col_name].values) / 8, 0, 1) * (current_temp_c_arr > data[sp_col_name].values)
        data[f'OAC_{coil_name_prefix}_개도율'] = np.clip(opening_rate_arr + np.random.normal(0,0.05,num_samples), 0, 1)

        temp_out_arr, abs_hum_out_arr, rh_out_arr, load_watts_arr = np.vectorize(simulate_oac_coil_process)(current_temp_c_arr, current_abs_hum_arr, data[sp_col_name].values, data[f'OAC_{coil_name_prefix}_개도율'].values, coil_name_prefix, oac_fan_cmh_values)
        data[f'OAC_{coil_name_prefix}_후단_온도'] = temp_out_arr
        data[f'OAC_{coil_name_prefix}_후단_습도'] = rh_out_arr

        if coil_name_prefix == '예냉': oac_precool_load_kw_series = pd.Series(load_watts_arr / 1000, index=data.index)
        if coil_name_prefix == '냉각': oac_cooling_load_kw_series = pd.Series(load_watts_arr / 1000, index=data.index)
        current_temp_c_arr, current_abs_hum_arr = temp_out_arr, abs_hum_out_arr

    current_rh_for_humidify = np.vectorize(calculate_rh_from_abs_humidity)(current_temp_c_arr, current_abs_hum_arr)
    opening_rate_humid = np.clip((data['OAC_가습_set_point'].values - current_rh_for_humidify) / 20, 0, 1) * (current_rh_for_humidify < data['OAC_가습_set_point'].values)
    data['OAC_가습_개도율'] = np.clip(opening_rate_humid + np.random.normal(0,0.05,num_samples), 0, 1)
    temp_out_arr, abs_hum_out_arr, rh_out_arr, _ = np.vectorize(simulate_oac_humidifier_process)(current_temp_c_arr, current_abs_hum_arr, data['OAC_가습_set_point'].values, data['OAC_가습_개도율'].values, oac_fan_cmh_values)

    data['OAC_토출_온도'] = temp_out_arr
    data['OAC_토출_노점_온도'] = np.vectorize(calculate_dew_point)(temp_out_arr, rh_out_arr)
    data['OAC_토출_압력'] = 101.0 + np.random.uniform(-0.5, 0.5, num_samples)

    chiller_ht_sys_load_kw = oac_precool_load_kw_series * num_oac_total_units
    chiller_lt_sys_load_kw = oac_cooling_load_kw_series * num_oac_total_units
    summer_indices_bool = data.index.month.isin([6,7,8])
    chiller_ht_sys_load_kw.loc[summer_indices_bool] *= np.random.uniform(1.5, 4.0, np.sum(summer_indices_bool))
    chiller_lt_sys_load_kw.loc[summer_indices_bool] *= np.random.uniform(1.5, 4.5, np.sum(summer_indices_bool))

    data['Chiller_HT_Sys_Load_kW_Internal'] = chiller_ht_sys_load_kw # 학습용으로 사용될 시스템 부하
    data['Chiller_LT_Sys_Load_kW_Internal'] = chiller_lt_sys_load_kw

    for type_prefix, sys_load_series, cap_kw, num_total_chillers in [
        ('고온', chiller_ht_sys_load_kw, CHILLER_CAPACITY_KW['high_temp'], NUM_CHILLERS_TOTAL['high_temp']),
        ('저온', chiller_lt_sys_load_kw, CHILLER_CAPACITY_KW['low_temp'], NUM_CHILLERS_TOTAL['low_temp'])
    ]:
        data[f'{type_prefix}냉동기_메인_차압_압력'] = 170 + sys_load_series * 0.02 + np.random.normal(0,10,num_samples)
        data[f'{type_prefix}냉동기_메인_차압_개도율'] = np.clip(30 + sys_load_series * 0.03, 10, 100)
        data[f'{type_prefix}냉동기_서브_차압_개도율'] = np.clip(data[f'{type_prefix}냉동기_메인_차압_개도율'] * np.random.uniform(0.7,1.0,num_samples) - 5, 5, 100)
        base_supply_temp = 7.0 if type_prefix == '고온' else 5.0
        data[f'{type_prefix}냉동기_메인_supply_온도'] = base_supply_temp - sys_load_series * 0.0005
        m_dot_water_total_kg_s = sys_load_series * 0.05 + 1
        cp_water_kj_kgk = 4.186
        delta_t_water = sys_load_series / (m_dot_water_total_kg_s * cp_water_kj_kgk + 1e-9)
        data[f'{type_prefix}냉동기_메인_return_온도'] = data[f'{type_prefix}냉동기_메인_supply_온도'] + delta_t_water
        data[f'{type_prefix}냉동기_메인_supply_압력'] = 490 + np.random.normal(0,20,num_samples)
        data[f'{type_prefix}냉동기_메인_return_압력'] = data[f'{type_prefix}냉동기_메인_supply_압력'] - data[f'{type_prefix}냉동기_메인_차압_압력']

        num_active_chillers_series = np.ceil(sys_load_series / cap_kw).clip(0, num_total_chillers).astype(int)
        data[f'{type_prefix}냉동기_활성대수_학습용'] = num_active_chillers_series # 학습용 가동대수

        for i in range(1, num_total_chillers + 1):
            is_active = num_active_chillers_series >= i
            data[f'{type_prefix}냉동기_{i}_supply_냉수온도'] = np.where(is_active, data[f'{type_prefix}냉동기_메인_supply_온도'] + np.random.normal(0,0.2,num_samples), np.nan)
            data[f'{type_prefix}냉동기_{i}_return_냉수온도'] = np.where(is_active, data[f'{type_prefix}냉동기_메인_return_온도'] + np.random.normal(0,0.2,num_samples), np.nan)
            data[f'{type_prefix}냉동기_{i}_supply_냉각수온도'] = np.nan
            data[f'{type_prefix}냉동기_{i}_return_냉각수온도'] = np.nan

    cop_ht_assumed = 4.0; cop_lt_assumed = 3.5
    ht_power_cons = chiller_ht_sys_load_kw / cop_ht_assumed
    lt_power_cons = chiller_lt_sys_load_kw / cop_lt_assumed
    ct_sys_load_kw_series = (chiller_ht_sys_load_kw + ht_power_cons) + (chiller_lt_sys_load_kw + lt_power_cons)
    data['CT_Sys_Load_kW_Internal'] = ct_sys_load_kw_series # 학습용

    num_active_ct_series = np.ceil(ct_sys_load_kw_series / COOLING_TOWER_CAPACITY_KW).clip(0, NUM_CTS_TOTAL).astype(int)
    data['냉각탑_활성대수_학습용'] = num_active_ct_series # 학습용 가동대수

    wet_bulb_approx = data['외기_온도'] - (100 - data['외기_습도']) / 5
    ct_approach = 3 + np.random.uniform(-0.5, 0.5, num_samples)
    ct_sys_supply_temp_c_series = np.maximum(wet_bulb_approx + ct_approach, 10.0)

    active_ct_divisor = num_active_ct_series.copy()
    active_ct_divisor[active_ct_divisor == 0] = 1 # 0으로 나누기 방지
    delta_t_ct_water = np.clip((ct_sys_load_kw_series / (active_ct_divisor * 100 + 1e-9)) * 0.1, 2, 10)
    ct_sys_return_temp_c_series = ct_sys_supply_temp_c_series + delta_t_ct_water

    for i in range(1, NUM_CTS_TOTAL + 1):
        is_active_ct = num_active_ct_series >= i
        data[f'냉각탑_{i}_supply_냉각수온도'] = np.where(is_active_ct, ct_sys_supply_temp_c_series + np.random.normal(0,0.1,num_samples), np.nan)
        data[f'냉각탑_{i}_return_냉각수온도'] = np.where(is_active_ct, ct_sys_return_temp_c_series + np.random.normal(0,0.1,num_samples), np.nan)
        data[f'냉각탑_{i}_supply_수조레벨'] = np.where(is_active_ct, 80 + np.random.uniform(-5,2,num_samples), np.nan)

    for type_prefix in ['고온', '저온']:
        num_total_chillers_for_type = NUM_CHILLERS_TOTAL['high_temp'] if type_prefix == '고온' else NUM_CHILLERS_TOTAL['low_temp']
        for i in range(1, num_total_chillers_for_type + 1):
            chiller_active_col_name = f'{type_prefix}냉동기_{i}_supply_냉수온도'
            is_chiller_active_series = ~data[chiller_active_col_name].isnull()

            # np.sum(is_chiller_active_series) 만큼의 랜덤 값 생성
            num_true = np.sum(is_chiller_active_series)
            if num_true > 0:
                supply_noise = np.random.normal(0,0.1,num_true)
                return_noise = np.random.normal(0,0.1,num_true)
                data.loc[is_chiller_active_series, f'{type_prefix}냉동기_{i}_supply_냉각수온도'] = ct_sys_supply_temp_c_series[is_chiller_active_series].values + supply_noise
                data.loc[is_chiller_active_series, f'{type_prefix}냉동기_{i}_return_냉각수온도'] = ct_sys_return_temp_c_series[is_chiller_active_series].values + return_noise
            else: # 혹시 is_chiller_active_series가 모두 False인 경우
                data[f'{type_prefix}냉동기_{i}_supply_냉각수온도'] = np.nan
                data[f'{type_prefix}냉동기_{i}_return_냉각수온도'] = np.nan


    return data.reset_index().rename(columns={'index': 'Datetime'})


dummy_df_user_spec = generate_dummy_data(periods=24*3, freq='60min')
print(f"\n사용자 명시 피처 기반 더미 데이터 생성 완료. 샘플 수: {len(dummy_df_user_spec)}")
# ... (이하 모델 클래스 정의, 학습, 시뮬레이션 코드는 이전 답변의 것을 가져와서 피처명 일치시키고 테스트 필요) ...
# --- 모델 클래스 및 시뮬레이션 함수는 이 새로운 더미 데이터의 컬럼명을 사용하도록 수정 필요 ---
# (이전 답변의 모델 클래스 정의를 가져와서 피처명 위주로 수정)

class OACModel:
    def __init__(self):
        self.models = {}
        self.is_trained = False
        self.feature_columns = {}

    def train(self, df):
        if df.empty: self.is_trained = False; return
        print("\nOAC 모델 학습 시작...")
        df_train = df.copy()

        for col in df_train.columns:
            if df_train[col].isnull().any():
                if pd.api.types.is_numeric_dtype(df_train[col]):
                    df_train[col] = df_train[col].fillna(df_train[col].median())
                else:
                    df_train[col] = df_train[col].fillna(df_train[col].mode()[0] if not df_train[col].mode().empty else "")

        oac_model_specs = {
            '예열_개도율': (['외기_온도', '외기_습도', 'OAC_예열_set_point', 'OAC_FAN_Hz'], 'OAC_예열_개도율'),
            '예냉_개도율': (['OAC_예열_후단_온도', 'OAC_예열_후단_습도', 'OAC_예냉_set_point', 'OAC_FAN_Hz'], 'OAC_예냉_개도율'),
            '냉각_개도율': (['OAC_예냉_후단_온도', 'OAC_예냉_후단_습도', 'OAC_냉각_set_point', 'OAC_FAN_Hz'], 'OAC_냉각_개도율'),
            '승온_개도율': (['OAC_냉각_후단_온도', 'OAC_냉각_후단_습도', 'OAC_승온_set_point', 'OAC_FAN_Hz'], 'OAC_승온_개도율'),
            '가습_개도율': (['OAC_승온_후단_온도', 'OAC_승온_후단_습도', 'OAC_가습_set_point', 'OAC_FAN_Hz'], 'OAC_가습_개도율'),
        }

        any_model_trained = False
        for model_name_suffix, (features, target) in oac_model_specs.items():
            model_full_name = f"OAC_{model_name_suffix}" # 모델 딕셔너리 키
            if not all(f_col in df_train.columns for f_col in features) or target not in df_train.columns:
                print(f"OAC {model_name_suffix} 모델 학습 불가: 필수 피처 {features} 또는 타겟 컬럼({target}) 누락")
                continue

            X = df_train[features].copy()
            self.feature_columns[model_full_name] = features
            y = df_train[target]

            # NaN 재확인 및 처리
            if X.isnull().values.any(): X = X.fillna(X.median())
            if y.isnull().any(): y = y.fillna(y.median())

            self.models[model_full_name] = RandomForestRegressor(n_estimators=30, random_state=42, max_depth=8, min_samples_leaf=10)
            self.models[model_full_name].fit(X, y)
            print(f"  OAC {model_name_suffix} 모델 학습 완료.")
            any_model_trained = True

        self.is_trained = any_model_trained
        if self.is_trained: print("OAC 모델 학습 완료.")
        else: print("OAC 모델 학습 실패: 학습된 하위 모델 없음.")


    def predict(self, ext_temp_c, ext_rh_percent, oac_sp_dict):
        oac_results = {'openings': {}, 'loads_kw': {},
                       'OAC_토출_온도': ext_temp_c,
                       'OAC_토출_노점_온도': calculate_dew_point(ext_temp_c, ext_rh_percent),
                       'OAC_토출_압력': 101.0} # 기본값

        if not self.is_trained or not self.models:
            print("경고: OAC 모델이 학습되지 않아 규칙/기본값으로 예측합니다.")
            # 모든 개도율을 0.1로, 부하도 임의값으로 설정 (예시)
            coil_types_for_openings = ['예열', '예냉', '냉각', '승온', '가습']
            for ct in coil_types_for_openings: oac_results['openings'][ct] = 0.1
            for ct in coil_types_for_openings: oac_results['loads_kw'][ct] = 5.0
            return oac_results


        current_temp_c = ext_temp_c
        current_rh_percent = ext_rh_percent
        current_abs_hum = calculate_absolute_humidity(current_temp_c, current_rh_percent)
        fan_hz = oac_sp_dict['OAC_FAN_Hz']
        fan_cmh = fan_hz * 1000

        coil_sequence_info = [
            ('예열', 'OAC_예열_set_point', 'OAC_예열_개도율'),
            ('예냉', 'OAC_예냉_set_point', 'OAC_예냉_개도율'),
            ('냉각', 'OAC_냉각_set_point', 'OAC_냉각_개도율'),
            ('승온', 'OAC_승온_set_point', 'OAC_승온_개도율')
        ]

        for coil_prefix, sp_key, model_key in coil_sequence_info:
            opening = 0.0
            if model_key in self.models and self.feature_columns.get(model_key):
                # 입력 피처: 이전 단계 출구 온도/습도, 현재 코일 SP, Fan_Hz
                # 첫 단계(예열)는 외기 사용
                in_temp = ext_temp_c if coil_prefix == '예열' else current_temp_c
                in_rh = ext_rh_percent if coil_prefix == '예열' else current_rh_percent

                # 모델 학습 시 사용한 피처 순서대로 입력 준비
                # 현재는 ['입구온도', '입구습도', 'SP', 'FanHz']로 가정
                input_features_for_model = [in_temp, in_rh, oac_sp_dict[sp_key], fan_hz]
                try:
                    pred_data = pd.DataFrame([input_features_for_model], columns=self.feature_columns[model_key])
                    opening = np.clip(self.models[model_key].predict(pred_data)[0], 0, 1)
                except Exception as e:
                    print(f"OAC {coil_prefix} 개도율 예측 오류: {e}. 피처: {self.feature_columns.get(model_key)}")
                    opening = 0.1 # 오류시 임의 개도율

            oac_results['openings'][coil_prefix] = opening
            temp_out, abs_hum_out, rh_out, load_watts = simulate_oac_coil_process(
                current_temp_c, current_abs_hum, oac_sp_dict[sp_key], opening, coil_prefix, fan_cmh
            )
            oac_results[f'OAC_{coil_prefix}_후단_온도'] = temp_out # 예측 결과에 후단 상태 저장 (참고용)
            oac_results[f'OAC_{coil_prefix}_후단_습도'] = rh_out
            oac_results['loads_kw'][coil_prefix] = load_watts / 1000
            current_temp_c, current_abs_hum, current_rh_percent = temp_out, abs_hum_out, rh_out

        # 가습 코일
        model_key_humid = 'OAC_가습_개도율'
        sp_key_humid = 'OAC_가습_set_point'
        opening_humid = 0.0
        if model_key_humid in self.models and self.feature_columns.get(model_key_humid):
            input_features_humid = [current_temp_c, current_rh_percent, oac_sp_dict[sp_key_humid], fan_hz]
            try:
                pred_data_humid = pd.DataFrame([input_features_humid], columns=self.feature_columns[model_key_humid])
                opening_humid = np.clip(self.models[model_key_humid].predict(pred_data_humid)[0], 0, 1)
            except Exception as e:
                print(f"OAC 가습 개도율 예측 오류: {e}. 피처: {self.feature_columns.get(model_key_humid)}")
                opening_humid = 0.1

        oac_results['openings']['가습'] = opening_humid
        temp_out, abs_hum_out, rh_out, load_watts = simulate_oac_humidifier_process(
            current_temp_c, current_abs_hum, oac_sp_dict[sp_key_humid], opening_humid, fan_cmh
        )
        oac_results['loads_kw']['가습'] = load_watts / 1000

        oac_results['OAC_토출_온도'] = temp_out
        # OAC_토출_노점_온도는 실제 토출 RH로 계산해야함
        oac_results['OAC_토출_노점_온도'] = calculate_dew_point(temp_out, rh_out)
        oac_results['OAC_토출_압력'] = 101.0

        return oac_results


class ChillerModel:
    def __init__(self, chiller_type_name):
        self.chiller_type_name = chiller_type_name
        self.model = RandomForestRegressor(n_estimators=30, random_state=42, max_depth=8, min_samples_leaf=10)
        self.capacity_kw = CHILLER_CAPACITY_KW['high_temp'] if chiller_type_name == '고온' else CHILLER_CAPACITY_KW['low_temp']
        self.num_total = NUM_CHILLERS_TOTAL['high_temp'] if chiller_type_name == '고온' else NUM_CHILLERS_TOTAL['low_temp']
        self.is_trained = False
        self.feature_name_ = '시스템_부하_kW'
        self.param_names_ = [f'{self.chiller_type_name}냉동기_메인_차압_압력', f'{self.chiller_type_name}냉동기_메인_차압_개도율', f'{self.chiller_type_name}냉동기_서브_차압_개도율', f'{self.chiller_type_name}냉동기_메인_supply_압력', f'{self.chiller_type_name}냉동기_메인_supply_온도', f'{self.chiller_type_name}냉동기_메인_return_압력', f'{self.chiller_type_name}냉동기_메인_return_온도']
        self.num_active_col_name_in_dummy = f'{self.chiller_type_name}냉동기_활성대수_학습용'


    def train(self, df):
        if df.empty: self.is_trained = False; return
        print(f"\n{self.chiller_type_name} 냉동기 모델 학습 시작 ...")
        load_col_from_dummy = 'Chiller_HT_Sys_Load_kW_Internal' if self.chiller_type_name == '고온' else 'Chiller_LT_Sys_Load_kW_Internal'

        if load_col_from_dummy not in df.columns:
            print(f"경고: {self.chiller_type_name} 냉동기 부하 컬럼({load_col_from_dummy}) 없음. 학습 불가.")
            self.is_trained = False; return

        X_df = df[[load_col_from_dummy]].rename(columns={load_col_from_dummy: self.feature_name_})
        y_cols_for_chiller = self.param_names_ + [self.num_active_col_name_in_dummy]

        if not all(col in df.columns for col in y_cols_for_chiller):
            missing_cols = [col for col in y_cols_for_chiller if col not in df.columns]
            print(f"경고: {self.chiller_type_name} 냉동기 학습 타겟 컬럼 부족: {missing_cols}. 학습 불가.")
            self.is_trained = False; return

        y_df = df[y_cols_for_chiller].copy()

        for col in X_df.columns: X_df[col] = X_df[col].fillna(X_df[col].median())
        for col in y_df.columns: y_df[col] = y_df[col].fillna(y_df[col].median())

        if X_df.isnull().values.any() or y_df.isnull().values.any():
             print(f"경고: {self.chiller_type_name} 냉동기 학습 데이터에 여전히 NaN 존재. 학습 중단.")
             self.is_trained = False; return

        self.model.fit(X_df, y_df)
        self.is_trained = True
        print(f"{self.chiller_type_name} 냉동기 모델 학습 완료.")

    def predict(self, system_total_load_kw):
        num_active_final = int(np.ceil(system_total_load_kw / self.capacity_kw).clip(0, self.num_total)) if self.capacity_kw > 0 and system_total_load_kw > 0 else 0
        default_params = {p_name: 0 for p_name in self.param_names_}

        if not self.is_trained:
            default_params['info'] = '모델 미학습 - 규칙 기반 가동대수 및 기본 파라미터'
            return default_params, num_active_final

        input_df = pd.DataFrame([[system_total_load_kw]], columns=[self.feature_name_])
        predicted_values_all = self.model.predict(input_df)[0]

        params = {self.param_names_[i]: predicted_values_all[i] for i in range(len(self.param_names_))}
        # 모델 예측 가동대수 (타겟에 포함시켰다면)
        num_active_from_model = int(round(predicted_values_all[len(self.param_names_)])) if len(predicted_values_all) > len(self.param_names_) else num_active_final
        num_active_final = np.clip(num_active_from_model, 0, self.num_total)

        return params, num_active_final


class CoolingTowerModel:
    def __init__(self):
        self.capacity_kw = COOLING_TOWER_CAPACITY_KW
        self.num_total = NUM_CTS_TOTAL
        self.model = RandomForestRegressor(n_estimators=30, random_state=42, max_depth=8, min_samples_leaf=10)
        self.is_trained = False
        self.feature_name_ = 'CT_Sys_Load_kW_Internal' # 더미 데이터의 냉각탑 시스템 부하 컬럼명
        self.target_names_ = ['CT_Sys_Supply_Temp_C', 'CT_Sys_Return_Temp_C', 'CT_Supply_TankLevel_Percent', '냉각탑_활성대수_학습용']

    def train(self, df):
        if df.empty or self.feature_name_ not in df.columns or not all(t in df.columns for t in self.target_names_):
            missing_cols = [t for t in self.target_names_ if t not in df.columns]
            print(f"경고: 냉각탑 모델 학습 데이터가 부족하거나 컬럼({self.feature_name_} 또는 타겟:{missing_cols})이 없습니다.")
            self.is_trained = False; return

        print(f"\n냉각탑 모델 학습 시작 ...")
        X_df = df[[self.feature_name_]].fillna(df[self.feature_name_].median())
        y_df = df[self.target_names_].copy()
        for col in y_df.columns: y_df[col] = y_df[col].fillna(y_df[col].median())

        if X_df.isnull().values.any() or y_df.isnull().values.any():
            print(f"경고: 냉각탑 학습 데이터에 여전히 NaN 존재. 학습 중단.")
            self.is_trained = False; return

        self.model.fit(X_df,y_df)
        self.is_trained = True
        print("냉각탑 모델 학습 완료.")

    def predict(self, ct_system_load_kw):
        params_predicted = {}
        num_active_final = 0

        if self.is_trained:
            input_df = pd.DataFrame([[ct_system_load_kw]], columns=[self.feature_name_])
            predicted_all = self.model.predict(input_df)[0]
            params_predicted[f'냉각탑_supply_냉각수온도'] = predicted_all[0]
            params_predicted[f'냉각탑_return_냉각수온도'] = predicted_all[1]
            params_predicted[f'냉각탑_supply_수조레벨'] = predicted_all[2]
            num_active_from_model = int(round(predicted_all[3]))
            num_active_final = np.clip(num_active_from_model, 0, self.num_total)
        else:
            num_active_final = int(np.ceil(ct_system_load_kw / self.capacity_kw).clip(0, self.num_total)) if self.capacity_kw > 0 and ct_system_load_kw > 0 else 0
            params_predicted[f'냉각탑_supply_냉각수온도'] = 25.0
            divisor = (num_active_final * 1000 * 4.186 + 1e-9) if num_active_final > 0 else (1 * 1000 * 4.186 + 1e-9)
            params_predicted[f'냉각탑_return_냉각수온도'] = params_predicted[f'냉각탑_supply_냉각수온도'] + (ct_system_load_kw / divisor )
            params_predicted[f'냉각탑_supply_수조레벨'] = 80.0
            params_predicted['info'] = '모델 미학습 - 규칙 기반'

        return params_predicted, num_active_final

# --- 3. 모델 학습 ---
if not dummy_df_user_spec.empty:
    oac_model = OACModel()
    oac_model.train(dummy_df_user_spec)
    chiller_lt_model = ChillerModel('저온')
    chiller_lt_model.train(dummy_df_user_spec)
    chiller_ht_model = ChillerModel('고온')
    chiller_ht_model.train(dummy_df_user_spec)
    ct_model = CoolingTowerModel()
    ct_model.train(dummy_df_user_spec)
else:
    print("더미 데이터가 비어있어 모델 학습을 건너<0xEB>니다.")
    oac_model=OACModel(); chiller_lt_model=ChillerModel('저온'); chiller_ht_model=ChillerModel('고온'); ct_model=CoolingTowerModel()

# --- 4. 시뮬레이션 실행 ---
def run_full_simulation(ext_temp_c, ext_rh_percent, oac_sp_values_dict, num_oac_total_units_sim=NUM_OAC_UNITS):
    print(f"\n--- 전체 시뮬레이션 시작 (OAC {num_oac_total_units_sim}대 가정): 외기 {ext_temp_c}°C, {ext_rh_percent}%RH ---")
    print(f"OAC SP: {oac_sp_values_dict}")

    if not all([oac_model.is_trained, chiller_lt_model.is_trained, chiller_ht_model.is_trained, ct_model.is_trained]):
        print("에러: 필수 모델 중 하나 이상이 학습되지 않았습니다. 시뮬레이션을 중단합니다.")
        # 학습 안된 모델에 대한 처리 (예: 기본값 사용) 또는 시뮬레이션 중단 결정
        return None

    single_oac_output = oac_model.predict(ext_temp_c, ext_rh_percent, oac_sp_values_dict)
    print("\n[단일 OAC 예측 결과]")
    for coil_name, opening_rate in single_oac_output['openings'].items(): print(f"  - OAC_{coil_name}_개도율: {opening_rate:.2f}")
    print("  단일 OAC 코일별 예상 부하 (kW):")
    for coil_name, load_val_kw in single_oac_output['loads_kw'].items(): print(f"    - OAC_{coil_name}_예상부하kW: {load_val_kw:.2f} kW")
    print(f"  단일 OAC 최종 토출: 온도={single_oac_output.get('OAC_토출_온도', 'N/A'):.1f}°C, 노점온도={single_oac_output.get('OAC_토출_노점_온도', 'N/A'):.1f}°C, 압력={single_oac_output.get('OAC_토출_압력', 'N/A'):.1f}kPa")

    sys_ht_load_kw = single_oac_output['loads_kw'].get('예냉', 0) * num_oac_total_units_sim
    sys_lt_load_kw = single_oac_output['loads_kw'].get('냉각', 0) * num_oac_total_units_sim
    print(f"\n  시스템 전체 고온냉동기 부하 (예냉 {num_oac_total_units_sim}대분): {sys_ht_load_kw:.2f} kW")
    print(f"  시스템 전체 저온냉동기 부하 (냉각 {num_oac_total_units_sim}대분): {sys_lt_load_kw:.2f} kW")

    print(f"\n[냉동기 시스템 예측 결과]")
    ht_params, ht_num_active = chiller_ht_model.predict(sys_ht_load_kw)
    print(f"  고온 냉동기 ({NUM_CHILLERS_TOTAL['high_temp']}대 중 {ht_num_active}대 가동 예측):")
    for p_name,val in ht_params.items(): print(f"    - {p_name}: {val:.2f}" if isinstance(val, (int, float)) else f"    - {p_name}: {val}")

    lt_params, lt_num_active = chiller_lt_model.predict(sys_lt_load_kw)
    print(f"  저온 냉동기 ({NUM_CHILLERS_TOTAL['low_temp']}대 중 {lt_num_active}대 가동 예측):")
    for p_name,val in lt_params.items(): print(f"    - {p_name}: {val:.2f}" if isinstance(val, (int, float)) else f"    - {p_name}: {val}")

    assumed_cop_ht = 4.0 ; assumed_cop_lt = 3.5
    ct_sys_load_kw = 0
    if sys_ht_load_kw > 0 and assumed_cop_ht > 0 : ct_sys_load_kw += sys_ht_load_kw * (1 + 1/assumed_cop_ht)
    if sys_lt_load_kw > 0 and assumed_cop_lt > 0 : ct_sys_load_kw += sys_lt_load_kw * (1 + 1/assumed_cop_lt)

    ct_params_predicted, ct_num_active = ct_model.predict(ct_sys_load_kw)
    print(f"\n[냉각탑 시스템 예측 결과 ({NUM_CTS_TOTAL}대 중 {ct_num_active}대 가동 예측)]")
    print(f"  냉각탑 시스템 총 부하 (열방출량): {ct_sys_load_kw:.2f} kW")
    # ct_params_predicted는 이미 요청하신 피처명으로 반환됨
    for p_name,val in ct_params_predicted.items(): print(f"    - {p_name}: {val:.2f}" if isinstance(val, (int, float)) else f"    - {p_name}: {val}")

    print("\n--- 시뮬레이션 종료 ---")
    return True

# --- 시나리오 설정 ---
oac_sp_summer = {'OAC_예열_set_point':18.0, 'OAC_예냉_set_point':18.0, 'OAC_냉각_set_point':12.0, 'OAC_승온_set_point':22.0, 'OAC_가습_set_point':50.0, 'OAC_FAN_Hz':55.0}
oac_sp_winter = {'OAC_예열_set_point':20.0, 'OAC_예냉_set_point':18.0, 'OAC_냉각_set_point':15.0, 'OAC_승온_set_point':24.0, 'OAC_가습_set_point':50.0, 'OAC_FAN_Hz':48.0}

if not dummy_df_user_spec.empty and oac_model.is_trained and chiller_lt_model.is_trained and chiller_ht_model.is_trained and ct_model.is_trained:
    print("\n"+"="*50+"\n 시나리오 1: 여름철 실행\n"+"="*50)
    run_full_simulation(ext_temp_c=30.0,ext_rh_percent=70.0,oac_sp_values_dict=oac_sp_summer)
    print("\n"+"="*50+"\n 시나리오 2: 겨울철 실행\n"+"="*50)
    run_full_simulation(ext_temp_c=0.0,ext_rh_percent=30.0,oac_sp_values_dict=oac_sp_winter)
else:
    print("모델 학습이 제대로 이루어지지 않아 시나리오 실행을 건너<0xEB>니다.")

<ipython-input-10-218c96ef433e>:197: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{type_prefix}냉동기_{i}_supply_냉각수온도'] = np.nan
<ipython-input-10-218c96ef433e>:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{type_prefix}냉동기_{i}_return_냉각수온도'] = np.nan
<ipython-input-10-218c96ef433e>:195: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a d


사용자 명시 피처 기반 더미 데이터 생성 완료. 샘플 수: 72

OAC 모델 학습 시작...
  OAC 예열_개도율 모델 학습 완료.
  OAC 예냉_개도율 모델 학습 완료.
  OAC 냉각_개도율 모델 학습 완료.
  OAC 승온_개도율 모델 학습 완료.
  OAC 가습_개도율 모델 학습 완료.
OAC 모델 학습 완료.

저온 냉동기 모델 학습 시작 ...
저온 냉동기 모델 학습 완료.

고온 냉동기 모델 학습 시작 ...
고온 냉동기 모델 학습 완료.
경고: 냉각탑 모델 학습 데이터가 부족하거나 컬럼(CT_Sys_Load_kW_Internal 또는 타겟:['CT_Sys_Supply_Temp_C', 'CT_Sys_Return_Temp_C', 'CT_Supply_TankLevel_Percent'])이 없습니다.
모델 학습이 제대로 이루어지지 않아 시나리오 실행을 건너<0xEB>니다.
